Purpose:
CM gradient optimisation of site similarity

In [6]:
# dependencies
from rdkit import Chem
from rdkit.Chem import AllChem, ChemicalForceFields
from rdkit.Chem import rdmolfiles

import qml
import ase
import ase.calculators
from ase.optimize import LBFGS

import numpy as np

import matplotlib.pyplot as plt
import pyscf
import pyscf.gto
import pyscf.qmmm
import pyscf.scf
import pyscf.dft
import pyscf.lib

In [22]:
def distance_bn_nb(charges, positions, idxA, idxB):
    size = 30
    q = charges.copy()
    q[[idxA, idxB]] = (5, 7)
    a = qml.representations.generate_coulomb_matrix(q, positions, size=size, sorting='row-norm')
    q = charges.copy()
    q[[idxA, idxB]] = (7, 5)
    b = qml.representations.generate_coulomb_matrix(q, positions, size=size, sorting='row-norm')
    return np.linalg.norm(a-b)
    
class CMGapCalc(ase.calculators.calculator.Calculator):
    implemented_properties = 'energy forces'.split()
    
    def __init__(self, pdbfile):
        self._pdbfile = pdbfile
        self._nevals = 0
        self._damping = 1000
    
    def _do_E(self, positions):
        ds = distance_bn_nb(c.nuclear_charges, positions, 4, 2)
        
        # add UFFs
        self._ffmol = Chem.MolFromPDBFile(self._pdbfile, removeHs=False)
        self._ffprop = ChemicalForceFields.MMFFGetMoleculeProperties(self._ffmol)
        self._ff = ChemicalForceFields.MMFFGetMoleculeForceField(self._ffmol, self._ffprop)
        conf = self._ffmol.GetConformer(0)
        for atom in range(len(positions)):
            conf.SetAtomPosition(atom, positions[atom])
        uffE = self._ff.CalcEnergy()
        
        return uffE/self._damping + ds
    
    def get_potential_energy(self, atoms=None, force_consistent=False):
        positions = atoms.get_positions()
        return self._do_E(positions)
    
    def get_forces(self, atoms=None):
        gradient = np.zeros(atoms.get_positions().shape)
        delta = 0.05
        positions = atoms.get_positions()
        for dim in range(3):
            for atom in range(len(atoms.get_positions())):
                fwd = positions.copy()
                fwd[atom, dim] += delta
                bwd = positions.copy()
                bwd[atom, dim] -= delta
                gradient[atom, dim] = (self._do_E(fwd) - self._do_E(bwd))/(2*delta)
        return -gradient
                
        

In [ ]:
c = qml.Compound('../../dsgdb9nsd_050720.xyz')
atm = ase.Atoms(numbers=c.nuclear_charges, positions=c.coordinates, calculator=CMGapCalc('../../dsgdb9nsd_050720.pdb'))
atmref = ase.Atoms(numbers=c.nuclear_charges, positions=c.coordinates, calculator=CMGapCalc('../../dsgdb9nsd_050720.pdb'))
dyn = LBFGS(atm, use_line_search=True,)
dyn.run(fmax=0.25)

       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 09:27:47       27.514951*      30.2537
LBFGS:    1 09:27:48       25.740770*      29.7575
LBFGS:    2 09:27:49       23.662487*      29.7389
LBFGS:    3 09:27:49       21.228340*      22.8563
LBFGS:    4 09:27:51       19.616820*     309.9451
